# Credit Risk Analysis

### NECESSARY IMPORTS

In [1]:
# import boto3
# import os
# import sys
import numpy as np
import pandas as pd
import random
random.seed=123
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split


# from dotenv import load_dotenv #for env variables
from imp import reload #for testing

from src import data_utils, config


C:\Users\Usuario\AppData\Local\Temp\ipykernel_19060\1820588173.py:16: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload #for testing


### GET DATA

In [2]:
reload(data_utils)
reload(config)

# get data
app_train, app_test, columns_description = data_utils.get_datasets()


c:\Users\Usuario\OneDrive\Documents\GitHub\credit-risk-analysis\src\data_utils.py:51: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  app_train = pd.read_csv(config.DATASET_TRAIN, delimiter='\t', encoding='latin1', header=None)


In [3]:
app_train.head()

,ID_CLIENT,CLERK_TYPE,PAYMENT_DAY,APPLICATION_SUBMISSION_TYPE,QUANT_ADDITIONAL_CARDS,POSTAL_ADDRESS_TYPE,SEX,MARITAL_STATUS,QUANT_DEPENDANTS,EDUCATION_LEVEL,...,FLAG_HOME_ADDRESS_DOCUMENT,FLAG_RG,FLAG_CPF,FLAG_INCOME_PROOF,PRODUCT,FLAG_ACSP_RECORD,AGE,RESIDENCIAL_ZIP_3,PROFESSIONAL_ZIP_3,TARGET_LABEL_BAD=1
0,1,C,5,Web,0,1,F,6,1,0,...,0,0,0,0,1,N,32,595,595,1
1,2,C,15,Carga,0,1,F,2,0,0,...,0,0,0,0,1,N,34,230,230,1
2,3,C,5,Web,0,1,F,2,0,0,...,0,0,0,0,1,N,27,591,591,0
3,4,C,20,Web,0,1,F,2,0,0,...,0,0,0,0,1,N,61,545,545,0
4,5,C,10,Web,0,1,M,2,0,0,...,0,0,0,0,1,N,48,235,235,1


In [4]:
# columns_description

### EDA

Before normalizing data, I want to see how the different columns are correlated with the column TARGET_LABEL_BAD=1'

In [5]:
# first let's copy the dataset
# app_normal = app_train.copy() # df created to normalize data
app_normal = app_train.loc[:, app_train.nunique() != 1].copy() # columns with a single value
app_normal.drop(columns=['ID_CLIENT'], inplace=True) # don't add value to our model

In [6]:
app_normal.corr()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_19060\2704048233.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  app_normal.corr()


,PAYMENT_DAY,POSTAL_ADDRESS_TYPE,MARITAL_STATUS,QUANT_DEPENDANTS,NACIONALITY,RESIDENCE_TYPE,MONTHS_IN_RESIDENCE,FLAG_EMAIL,PERSONAL_MONTHLY_INCOME,OTHER_INCOMES,...,PERSONAL_ASSETS_VALUE,QUANT_CARS,MONTHS_IN_THE_JOB,PROFESSION_CODE,OCCUPATION_TYPE,MATE_PROFESSION_CODE,EDUCATION_LEVEL,PRODUCT,AGE,TARGET_LABEL_BAD=1
PAYMENT_DAY,1.000000,0.011134,-0.042197,0.007054,0.007192,0.008132,-0.031131,0.019423,-0.006796,-0.002590,...,-0.006425,-0.022499,0.003107,-0.027909,0.106362,-0.001377,0.006458,0.006373,-0.153042,0.070633
POSTAL_ADDRESS_TYPE,0.011134,1.000000,-0.003475,0.012115,0.019099,0.024265,-0.019871,-0.004576,0.027271,0.004062,...,0.034770,-0.001533,0.000616,0.011404,0.009462,-0.001813,0.020164,0.002974,-0.030579,-0.002423
MARITAL_STATUS,-0.042197,-0.003475,1.000000,0.037646,0.049219,0.000310,0.061487,-0.022849,-0.000311,0.005852,...,0.012739,0.011005,0.004016,-0.003760,-0.072520,-0.084623,0.005377,0.010545,0.237030,-0.030380
QUANT_DEPENDANTS,0.007054,0.012115,0.037646,1.000000,-0.072128,-0.007526,-0.039378,0.006691,0.008035,0.002430,...,0.016049,0.025209,0.007597,0.020347,0.022296,0.110612,0.060360,0.009623,-0.037415,0.012157
NACIONALITY,0.007192,0.019099,0.049219,-0.072128,1.000000,-0.001562,0.173154,-0.096312,-0.003406,0.008126,...,0.011621,-0.247115,0.004618,-0.121835,0.032621,-0.444238,0.113460,-0.006574,0.001579,-0.000952
RESIDENCE_TYPE,0.008132,0.024265,0.000310,-0.007526,-0.001562,1.000000,-0.093517,0.005490,-0.004802,-0.002543,...,-0.012420,-0.007703,0.002481,0.040090,-0.010574,0.021213,0.028257,0.001629,-0.106723,0.019168
MONTHS_IN_RESIDENCE,-0.031131,-0.019871,0.061487,-0.039378,0.173154,-0.093517,1.000000,-0.022810,0.012685,0.016208,...,0.031802,-0.043472,0.014715,-0.027821,-0.011008,-0.319603,0.035550,-0.020528,0.223815,-0.029336
FLAG_EMAIL,0.019423,-0.004576,-0.022849,0.006691,-0.096312,0.005490,-0.022810,1.000000,-0.006100,-0.002531,...,-0.001392,-0.697017,0.012066,0.258016,0.124020,0.599763,0.151961,-0.004697,-0.018876,-0.008067
PERSONAL_MONTHLY_INCOME,-0.006796,0.027271,-0.000311,0.008035,-0.003406,-0.004802,0.012685,-0.006100,1.000000,0.000032,...,0.008066,0.006403,0.000095,-0.006259,0.001560,0.000006,0.007097,-0.001967,0.004820,0.002819
OTHER_INCOMES,-0.002590,0.004062,0.005852,0.002430,0.008126,-0.002543,0.016208,-0.002531,0.000032,1.000000,...,0.003353,0.000717,0.003335,0.002557,-0.003994,-0.037819,0.071309,-0.000704,0.012694,0.004442


In [7]:
cor_target = app_normal.corr()
cor_target.iloc[:]['TARGET_LABEL_BAD=1']


C:\Users\Usuario\AppData\Local\Temp\ipykernel_19060\1977501773.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  cor_target = app_normal.corr()


PAYMENT_DAY                       0.070633
POSTAL_ADDRESS_TYPE              -0.002423
MARITAL_STATUS                   -0.030380
QUANT_DEPENDANTS                  0.012157
NACIONALITY                      -0.000952
RESIDENCE_TYPE                    0.019168
MONTHS_IN_RESIDENCE              -0.029336
FLAG_EMAIL                       -0.008067
PERSONAL_MONTHLY_INCOME           0.002819
OTHER_INCOMES                     0.004442
FLAG_VISA                        -0.005398
FLAG_MASTERCARD                  -0.017658
FLAG_DINERS                       0.002240
FLAG_AMERICAN_EXPRESS             0.000337
FLAG_OTHER_CARDS                  0.001410
QUANT_BANKING_ACCOUNTS            0.011997
QUANT_SPECIAL_BANKING_ACCOUNTS    0.011997
PERSONAL_ASSETS_VALUE            -0.003886
QUANT_CARS                        0.011706
MONTHS_IN_THE_JOB                -0.007073
PROFESSION_CODE                  -0.011593
OCCUPATION_TYPE                   0.040216
MATE_PROFESSION_CODE             -0.034505
EDUCATION_L

- As we can see, not all columns are in the correlation table, it is mainly because we have some object type columns.
- Let's normalize and transform those columns in a number or category type.
- Some columns will be deleted, due to the lack of information (missing/empty/null values)

In [8]:
app_normal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 44 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   PAYMENT_DAY                     50000 non-null  int64  
 1   APPLICATION_SUBMISSION_TYPE     50000 non-null  object 
 2   POSTAL_ADDRESS_TYPE             50000 non-null  int64  
 3   SEX                             50000 non-null  object 
 4   MARITAL_STATUS                  50000 non-null  int64  
 5   QUANT_DEPENDANTS                50000 non-null  int64  
 6   STATE_OF_BIRTH                  50000 non-null  object 
 7   CITY_OF_BIRTH                   50000 non-null  object 
 8   NACIONALITY                     50000 non-null  int64  
 9   RESIDENCIAL_STATE               50000 non-null  object 
 10  RESIDENCIAL_CITY                50000 non-null  object 
 11  RESIDENCIAL_BOROUGH             50000 non-null  object 
 12  FLAG_RESIDENCIAL_PHONE          

In [9]:
col_object = app_normal.select_dtypes(exclude="number").columns
print(col_object)

Index(['APPLICATION_SUBMISSION_TYPE', 'SEX', 'STATE_OF_BIRTH', 'CITY_OF_BIRTH',
       'RESIDENCIAL_STATE', 'RESIDENCIAL_CITY', 'RESIDENCIAL_BOROUGH',
       'FLAG_RESIDENCIAL_PHONE', 'RESIDENCIAL_PHONE_AREA_CODE', 'COMPANY',
       'PROFESSIONAL_STATE', 'PROFESSIONAL_CITY', 'PROFESSIONAL_BOROUGH',
       'FLAG_PROFESSIONAL_PHONE', 'PROFESSIONAL_PHONE_AREA_CODE',
       'RESIDENCIAL_ZIP_3', 'PROFESSIONAL_ZIP_3'],
      dtype='object')


- CHANGES DONE TO CATEGORICAL COLUMNS

In [10]:
app_normal.groupby('SEX')['SEX'].value_counts().sort_values()

SEX  SEX
               17
N    N         48
M    M      19130
F    F      30805
Name: SEX, dtype: int64

In [11]:
# 'APPLICATION_SUBMISSION_TYPE', I will assume 0 is a typo mistake for Carga, and web is fully ok, so 0 will be change to 'Carga'
app_normal['APPLICATION_SUBMISSION_TYPE'] = np.where(app_normal['APPLICATION_SUBMISSION_TYPE']=='Web', 'Web', 'Carga')
app_normal['APPLICATION_SUBMISSION_TYPE'] = app_normal['APPLICATION_SUBMISSION_TYPE'].astype('category')

#We can change the data of the variable "sex" having value N and empty space by the % of values set in M and F

# 'SEX' deleted missing rows, changed to categorical

# app_normal = app_normal.drop(app_normal[(app_normal['SEX'] == ' ') | (app_normal['SEX'] == 'N')].index)
# app_normal['SEX'] = np.where(app_normal['SEX']=='M', 0, 1)
# app_normal['SEX'] = app_normal['SEX'].astype('category')

# 1. Analizar el porcentaje de las categorías que contiene la columna.
# Suponiendo que 'SEX' contiene valores 'femenino', 'masculino' y NaN o algún otro valor considerado como "fuera de categoría".

# Obtén la distribución de los datos, excluyendo los valores fuera de categoría
app_normal = app_normal.copy()
valid_values = ['M', 'F']  # las categorías válidas
distribution = app_normal[app_normal['SEX'].isin(valid_values)]['SEX'].value_counts(normalize=True)
print = distribution
# 2. Tomar los valores fuera de categoría y reescribirlos de acuerdo al porcentaje de valores que se analizaron inicialmente.

# Identifica los valores que están fuera de categoría.
out_of_category = ~app_normal['SEX'].isin(valid_values)

# Imputar los valores fuera de categoría de acuerdo a la distribución de los datos existentes.
app_normal.loc[out_of_category, 'SEX'] = np.random.choice(distribution.index, size=len(app_normal[out_of_category]), p=distribution.values)
app_normal['SEX'] = app_normal['SEX'].astype('category')

# residential state converted to categorical, has no missing values
app_normal['RESIDENCIAL_STATE']=app_normal['RESIDENCIAL_STATE'].astype('category')

# converted to categorical, has no missing values
app_normal['FLAG_RESIDENCIAL_PHONE'] = app_normal['FLAG_RESIDENCIAL_PHONE'].astype('category')

#  converted to categorical, has no missing values
app_normal['COMPANY'] = app_normal['COMPANY'].astype('category')

# converted to categorical, has no missing values
app_normal['FLAG_PROFESSIONAL_PHONE'] = app_normal['FLAG_RESIDENCIAL_PHONE'].astype('category')


# not relevant, lot's of missing values or too many different arguments
app_normal.drop(columns=['STATE_OF_BIRTH', # too many empty values
              'CITY_OF_BIRTH', 'RESIDENCIAL_CITY', 'RESIDENCIAL_BOROUGH', # too much work for normalizing
              'RESIDENCIAL_PHONE_AREA_CODE', # too many empty values], inplace=True) 
              'PROFESSIONAL_STATE', # too many empty values
              'PROFESSIONAL_CITY', # uncountable values to analize
              'PROFESSIONAL_BOROUGH', # too many empty values
              'PROFESSIONAL_PHONE_AREA_CODE', # too many empty values
              'RESIDENCIAL_ZIP_3', # uncountable values to analize
              'PROFESSIONAL_ZIP_3'], # uncountable values to analize
              inplace=True)

Now let's work with numerical columns

In [12]:
col_numbers = col_object = app_normal.select_dtypes(exclude=["object", "category"]).columns
col_numbers

Index(['PAYMENT_DAY', 'POSTAL_ADDRESS_TYPE', 'MARITAL_STATUS',
       'QUANT_DEPENDANTS', 'NACIONALITY', 'RESIDENCE_TYPE',
       'MONTHS_IN_RESIDENCE', 'FLAG_EMAIL', 'PERSONAL_MONTHLY_INCOME',
       'OTHER_INCOMES', 'FLAG_VISA', 'FLAG_MASTERCARD', 'FLAG_DINERS',
       'FLAG_AMERICAN_EXPRESS', 'FLAG_OTHER_CARDS', 'QUANT_BANKING_ACCOUNTS',
       'QUANT_SPECIAL_BANKING_ACCOUNTS', 'PERSONAL_ASSETS_VALUE', 'QUANT_CARS',
       'MONTHS_IN_THE_JOB', 'PROFESSION_CODE', 'OCCUPATION_TYPE',
       'MATE_PROFESSION_CODE', 'EDUCATION_LEVEL', 'PRODUCT', 'AGE',
       'TARGET_LABEL_BAD=1'],
      dtype='object')

In [13]:
#'PAYMENT_DAY' : split every 15 days, converted to category
app_normal['PAYMENT_DAY_C'] = np.where(app_normal['PAYMENT_DAY'] < 15, '1-14', '15-30')
app_normal['PAYMENT_DAY_C'] = app_normal['PAYMENT_DAY_C'].astype('category')

# changed to categorical
app_normal['POSTAL_ADDRESS_TYPE'] = app_normal['POSTAL_ADDRESS_TYPE'].astype('category')

# normalize marital status adding one more status equals to 3, and deleting the remaining, changed to categorical
marital_array = app_normal['MARITAL_STATUS'].unique() # get all unique values
marital_array = np.delete(marital_array, np.where(marital_array == 1)) # delete values = 1
marital_array = np.delete(marital_array, np.where(marital_array == 2)) # delete values = 2
app_normal['MARITAL_STATUS'] = app_normal['MARITAL_STATUS'].replace(marital_array, 3) # change all different values to 0
app_normal['MARITAL_STATUS'] = app_normal['MARITAL_STATUS'].astype('category')

# 'QUANT_DEPENDANTS' : values, 0 for not dependants and 1 if have dependants, changed to categorical
app_normal['HAS_DEPENDANTS'] = [1 if dep > 0 else 0 for dep in app_normal['QUANT_DEPENDANTS']] # 0 AND 1
app_normal['HAS_DEPENDANTS'] = app_normal['HAS_DEPENDANTS'].astype('category')

# 'NACIONALITY': changed as categoty, it can also be reduced to 2 types, 1 and 2 (adding 0 to 2)
app_normal['NACIONALITY'] = app_normal['NACIONALITY'].astype('category')

# fill null values with 0, divided in 3 categories (1-6 months, 1 year, +1 year), changed to category
# app_normal['MONTHS_IN_RESIDENCE'] = app_normal['MONTHS_IN_RESIDENCE'].fillna(0)
# app_normal['MONTHS_IN_RESIDENCE'] = np.where(app_normal['MONTHS_IN_RESIDENCE'] < 6, '6 months',
#             np.where(app_normal['MONTHS_IN_RESIDENCE'] < 13, '1 year', '+1 year')) # divided in 6m, 1y, +1y
# app_normal['MONTHS_IN_RESIDENCE'] = app_normal['MONTHS_IN_RESIDENCE'].astype('category')

def categorize_residence(months):
    if months <= 12:
        return '0-1 año'
    elif 12 < months <= 24:
        return '1-2 años'
    else:
        return '> 2 años'

# Aplicar la función a la columna
app_normal['MONTHS_IN_RESIDENCE'] = app_normal['MONTHS_IN_RESIDENCE'].apply(categorize_residence)

# Obtén la distribución de los datos, excluyendo los valores NaN
valid_values = ['0-1 año', '1-2 años', '> 2 años']  # las categorías válidas
distribution = app_normal[app_normal['MONTHS_IN_RESIDENCE'].isin(valid_values)]['MONTHS_IN_RESIDENCE'].value_counts(normalize=True)

# Identifica los valores que están NaN.
missing = app_normal['MONTHS_IN_RESIDENCE'].isnull()

# Imputar los valores NaN de acuerdo a la distribución de los datos existentes.
app_normal.loc[missing, 'MONTHS_IN_RESIDENCE'] = np.random.choice(distribution.index, size=len(app_normal[missing]), p=distribution.values)
# Set this column as category type
app_normal['MONTHS_IN_RESIDENCE'] = app_normal['MONTHS_IN_RESIDENCE'].astype('category')

# FLAG_EMAIL: changed to category
app_normal['FLAG_EMAIL'] = app_normal['FLAG_EMAIL'].astype('category')

# 'PERSONAL_MONTHLY_INCOME_LEVEL': 5 different ranges (<500, 500-1000, 1000-1500, 1500-2000, +2000), changed to category
#app_normal['PERSONAL_MONTHLY_INCOME'] = app_normal['PERSONAL_MONTHLY_INCOME_LEVEL'].astype('float')
app_normal['PERSONAL_MONTHLY_INCOME_LEVEL'] = np.where(app_normal['PERSONAL_MONTHLY_INCOME'] < 501, '< 500',
            np.where(app_normal['PERSONAL_MONTHLY_INCOME'] < 1001, '500 - 1000',
            np.where(app_normal['PERSONAL_MONTHLY_INCOME'] < 1500, '1000 - 1500',
            np.where(app_normal['PERSONAL_MONTHLY_INCOME'] < 2000, '1500 - 2000', '> 2000'))))
app_normal['PERSONAL_MONTHLY_INCOME_LEVEL'] = app_normal['PERSONAL_MONTHLY_INCOME_LEVEL'].astype('category')

# 'OTHER_INCOMES' with 2 values, Y, N, changed to category
app_normal['EXTRA_INCOME'] = ['Y' if income > 0 else 'N' for income in app_normal['OTHER_INCOMES']]
app_normal['EXTRA_INCOME'] = app_normal['EXTRA_INCOME'].astype('category')

# 'FLAG_VISA', 'FLAG_MASTERCARD', 'FLAG_DINERS', 'FLAG_AMERICAN_EXPRESS', 'FLAG_OTHER_CARDS'
# changed for FLAG_CARD, has 2 values Y, N, type category
app_normal['FLAG_CARDS'] = np.where(app_normal['FLAG_VISA']>0, 'Y', 
            np.where(app_normal['FLAG_MASTERCARD']>0, 'Y',
            np.where(app_normal['FLAG_DINERS']>0, 'Y',
            np.where(app_normal['FLAG_AMERICAN_EXPRESS']>0, 'Y',
            np.where(app_normal['FLAG_OTHER_CARDS']>0, 'Y', 'N')))))
app_normal['FLAG_CARDS'] = app_normal['FLAG_CARDS'].astype('category')

# QUANT_BANKING_ACCOUNTS: set 2 values, Y, N, changed to category
app_normal['HAS_BANKING_ACCOUNTS'] = np.where(app_normal['QUANT_BANKING_ACCOUNTS']>0, 'Y', 'N')
app_normal['HAS_BANKING_ACCOUNTS'] = app_normal['HAS_BANKING_ACCOUNTS'].astype('category')

# QUANT_SPECIAL_BANKING_ACCOUNTS: set 2 values, Y, N, changed to category
app_normal['HAS_SPECIAL_BANKING_ACCOUNTS'] = np.where(app_normal['QUANT_SPECIAL_BANKING_ACCOUNTS']>0, 'Y', 'N')
app_normal['HAS_SPECIAL_BANKING_ACCOUNTS'] = app_normal['HAS_SPECIAL_BANKING_ACCOUNTS'].astype('category')

# PERSONAL_ASSETS_VALUE: set 2 values, Y, N, changed to category
app_normal['PERSONAL_ASSETS_VALUE'] = np.where(app_normal['PERSONAL_ASSETS_VALUE']>0, 'Y', 'N')
app_normal['PERSONAL_ASSETS_VALUE'] = app_normal['PERSONAL_ASSETS_VALUE'].astype('category')

# QUANT_CARS: set 2 values, Y, N, changed to category
app_normal['HAS_CARS'] = np.where(app_normal['QUANT_CARS']>0, 'Y', 'N')
app_normal['HAS_CARS'] = app_normal['QUANT_CARS'].astype('category')

# OCCUPATION_TYPE replace nan for weighted values, changed to category
# Obtén la distribución de los datos, excluyendo los valores NaN
valid_values = [1 , 2 , 3 , 4, 5, 0]  # las categorías válidas
distribution = app_normal[app_normal['OCCUPATION_TYPE'].isin(valid_values)]['OCCUPATION_TYPE'].value_counts(normalize=True)

# Identifica los valores que están NaN.
missing = app_normal['OCCUPATION_TYPE'].isnull()

# Imputar los valores NaN de acuerdo a la distribución de los datos existentes.
app_normal.loc[missing, 'OCCUPATION_TYPE'] = np.random.choice(distribution.index, size=len(app_normal[missing]), p=distribution.values)
app_normal['OCCUPATION_TYPE'] = app_normal['OCCUPATION_TYPE'].astype('category')

# EDUCATION_LEVEL replace nan for weighted values, changed to category
# Obtén la distribución de los datos, excluyendo los valores NaN
valid_values = [1 , 2 , 3 , 4, 5, 0]  # las categorías válidas
distribution = app_normal[app_normal['EDUCATION_LEVEL'].isin(valid_values)]['EDUCATION_LEVEL'].value_counts(normalize=True)

# Identifica los valores que están NaN.
missing = app_normal['EDUCATION_LEVEL'].isnull()

# Imputar los valores NaN de acuerdo a la distribución de los datos existentes.
app_normal.loc[missing, 'EDUCATION_LEVEL'] = np.random.choice(distribution.index, size=len(app_normal[missing]), p=distribution.values)
app_normal['EDUCATION_LEVEL'] = app_normal['EDUCATION_LEVEL'].astype('category')

# PRODUCT changed to category
app_normal['PRODUCT'] = app_normal['PRODUCT'].astype('category')


# AGE changed to category in ranges of 10 years from 0-20, 20-30, 30-40, 40-50, 50-60, >60
def categorize_age(age):
    if age <= 20:
        return '0-20 años'
    elif 20 < age <= 30:
        return '20-30 años'
    elif 30 < age <= 40:
        return '30-40 años'
    elif 40 < age <= 50:
        return '40-50 años'
    elif 50 < age <= 60:
        return '50-60 años'
    else:
        return '> 60 años'

# Aplicar la función a la columna
app_normal['AGE'] = app_normal['AGE'].apply(categorize_age)

# Obtén la distribución de los datos, excluyendo los valores NaN
valid_values = ['0-20 años', '20-30 años', '30-40 años' , '40-50 años' , '50-60 años' ,'> 60 años']  # las categorías válidas
distribution = app_normal[app_normal['AGE'].isin(valid_values)]['AGE'].value_counts(normalize=True)

# Identifica los valores que están NaN.
missing = app_normal['AGE'].isnull()

# Imputar los valores NaN de acuerdo a la distribución de los datos existentes.
app_normal.loc[missing, 'AGE'] = np.random.choice(distribution.index, size=len(app_normal[missing]), p=distribution.values)
#set this column as category type
app_normal['AGE'] = app_normal['AGE'].astype('category')

# NOT FINISHED YET



app_normal.drop(columns=['PAYMENT_DAY', # replaced by PAYMENT_DAY_C
                         'QUANT_DEPENDANTS', # replaced with HAS_DEPENDANTS
                         'RESIDENCE_TYPE', # has too many empty rows
                         'OTHER_INCOMES', # replaced with OTHER_INCOMES, 
                         'FLAG_VISA', 'FLAG_MASTERCARD', 'FLAG_DINERS', # replaced with FLAG_CARDS
                         'FLAG_AMERICAN_EXPRESS', 'FLAG_OTHER_CARDS', # replaced with FLAG_CARDS
                         'QUANT_BANKING_ACCOUNTS', # replaced with HAS_BANKING_ACCOUNTS 
                         'QUANT_SPECIAL_BANKING_ACCOUNTS', # replaced with HAS_SPECIAL_BANKING_ACCOUNTS
                         'MONTHS_IN_THE_JOB', # 90% of values = 0
                         'QUANT_CARS' , # Was Changed for 'HAS_CARS'
                         'PROFESSION_CODE', # Without relevant information abot this codes
                         'MATE_PROFESSION_CODE', # Without relevant information abot this codes
                         'PERSONAL_MONTHLY_INCOME' # Was changed for 'PERSONAL_MONTHLY_INCOMES_LEVEL'
                         ], inplace=True)

# app_normal.drop(columns=[curr_col_name],inplace=True)
# number_field_name.remove(curr_col_name)

In [14]:
col_object = app_normal.select_dtypes(exclude="number").columns
col_object.to_list()

['APPLICATION_SUBMISSION_TYPE',
 'POSTAL_ADDRESS_TYPE',
 'SEX',
 'MARITAL_STATUS',
 'NACIONALITY',
 'RESIDENCIAL_STATE',
 'FLAG_RESIDENCIAL_PHONE',
 'MONTHS_IN_RESIDENCE',
 'FLAG_EMAIL',
 'PERSONAL_ASSETS_VALUE',
 'COMPANY',
 'FLAG_PROFESSIONAL_PHONE',
 'OCCUPATION_TYPE',
 'EDUCATION_LEVEL',
 'PRODUCT',
 'AGE',
 'PAYMENT_DAY_C',
 'HAS_DEPENDANTS',
 'PERSONAL_MONTHLY_INCOME_LEVEL',
 'EXTRA_INCOME',
 'FLAG_CARDS',
 'HAS_BANKING_ACCOUNTS',
 'HAS_SPECIAL_BANKING_ACCOUNTS',
 'HAS_CARS']

In [15]:
app_normal.head()

,APPLICATION_SUBMISSION_TYPE,POSTAL_ADDRESS_TYPE,SEX,MARITAL_STATUS,NACIONALITY,RESIDENCIAL_STATE,FLAG_RESIDENCIAL_PHONE,MONTHS_IN_RESIDENCE,FLAG_EMAIL,PERSONAL_ASSETS_VALUE,...,AGE,TARGET_LABEL_BAD=1,PAYMENT_DAY_C,HAS_DEPENDANTS,PERSONAL_MONTHLY_INCOME_LEVEL,EXTRA_INCOME,FLAG_CARDS,HAS_BANKING_ACCOUNTS,HAS_SPECIAL_BANKING_ACCOUNTS,HAS_CARS
0,Web,1,F,3,1,RN,Y,1-2 años,1,N,...,30-40 años,1,1-14,1,500 - 1000,N,Y,N,N,0
1,Carga,1,F,2,1,RJ,Y,0-1 año,1,N,...,30-40 años,1,15-30,0,500 - 1000,N,N,N,N,0
2,Web,1,F,2,1,RN,Y,> 2 años,1,N,...,20-30 años,0,1-14,0,< 500,N,N,N,N,0
3,Web,1,F,2,1,PE,N,> 2 años,1,N,...,> 60 años,0,15-30,0,< 500,N,N,N,N,0
4,Web,1,M,2,1,RJ,Y,0-1 año,1,N,...,40-50 años,1,1-14,0,1000 - 1500,N,N,N,N,0


In [16]:
#app_normal.groupby(['HAS_DEPENDANTS','PERSONAL_MONTHLY_INCOME_LEVEL'])['TARGET_LABEL_BAD=1'].value_counts().sort_values()

In [17]:
app_normal.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 25 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   APPLICATION_SUBMISSION_TYPE    50000 non-null  category
 1   POSTAL_ADDRESS_TYPE            50000 non-null  category
 2   SEX                            50000 non-null  category
 3   MARITAL_STATUS                 50000 non-null  category
 4   NACIONALITY                    50000 non-null  category
 5   RESIDENCIAL_STATE              50000 non-null  category
 6   FLAG_RESIDENCIAL_PHONE         50000 non-null  category
 7   MONTHS_IN_RESIDENCE            50000 non-null  category
 8   FLAG_EMAIL                     50000 non-null  category
 9   PERSONAL_ASSETS_VALUE          50000 non-null  category
 10  COMPANY                        50000 non-null  category
 11  FLAG_PROFESSIONAL_PHONE        50000 non-null  category
 12  OCCUPATION_TYPE                5

Despues de hacer EDA  al dataset de entrenamiento las columnas quedaron de la siguiente manera:

Categoricos:





'QUANT_BANKING_ACCOUNTS' tiene 3 valores (2=14, 1=17864, 0=32122), se podria categorizar en si tiene o no tiene cuenta

In [18]:
#split into train, val and test
TARGET = app_normal['TARGET_LABEL_BAD=1']
app_normal_train = app_normal.drop(columns=['TARGET_LABEL_BAD=1'])

X_train, X_test, y_train, y_test = train_test_split(app_normal_train, TARGET, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [19]:
    # Taking the columns that contain objects.
    category_columns = X_train.select_dtypes(include=['category']).columns

    # Initializing the encoders
    binary_encoder = OrdinalEncoder()
    onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

    # Filtering the dataset.
    aux_dataframe = X_train[category_columns].copy()
    mask_2 = (aux_dataframe.nunique() == 2).values
    cat_2 = aux_dataframe.loc[:, mask_2].columns
    mask_gt_2 = (aux_dataframe.nunique() > 2).values
    cat_gt_2 = aux_dataframe.loc[:, mask_gt_2].columns

    # Setting the tranform.
    ct = ColumnTransformer(transformers=[
        ('binary', binary_encoder, cat_2.tolist()),
        ('onehot', onehot_encoder, cat_gt_2.tolist())
    ], remainder='passthrough')

    # Setting the dtype output
    # ct.set_output(transform="pandas")

    # fitting the ecoder.
    ct.fit(X_train)

    # Applying the transformation.
    app_normal_train_encode = ct.transform(X_train)

    working_val_df = ct.transform(X_val)

    working_test_df = ct.transform(X_test)

c:\Users\Usuario\anaconda3\envs\fabenv\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [26]:
app_normal_train_encode

array([[1., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

HASTA ACÁ LLEGAMOS HOY

In [20]:
# def preprocess_data(
#     train_df: pd.DataFrame, val_df: pd.DataFrame, test_df: pd.DataFrame
# ) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
#     """
#     Pre processes data for modeling. Receives train, val and test dataframes
#     and returns numpy ndarrays of cleaned up dataframes with feature engineering
#     already performed.

#     Arguments:
#         train_df : pd.DataFrame
#         val_df : pd.DataFrame
#         test_df : pd.DataFrame

#     Returns:
#         train : np.ndarrary
#         val : np.ndarrary
#         test : np.ndarrary
#     """
#     # Print shape of input data
#     print("Input train data shape: ", train_df.shape)
#     print("Input val data shape: ", val_df.shape)
#     print("Input test data shape: ", test_df.shape, "\n")

#     # Make a copy of the dataframes
#     working_train_df = train_df.copy()
#     working_val_df = val_df.copy()
#     working_test_df = test_df.copy()

#     # 1. Correct outliers/anomalous values in numerical
#     # columns (`DAYS_EMPLOYED` column).
#     working_train_df["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace=True)
#     working_val_df["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace=True)
#     working_test_df["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace=True)

#     # 2. TODO Encode string categorical features (dytpe `object`):
#     #     - If the feature has 2 categories encode using binary encoding,
#     #       please use `sklearn.preprocessing.OrdinalEncoder()`. Only 4 columns
#     #       from the dataset should have 2 categories.
#     #     - If it has more than 2 categories, use one-hot encoding, please use
#     #       `sklearn.preprocessing.OneHotEncoder()`. 12 columns
#     #       from the dataset should have more than 2 categories.

#     # Take into account that:
#     #   - You must apply this to the 3 DataFrames (working_train_df, working_val_df,
#     #     working_test_df).
#     #   - In order to prevent overfitting and avoid Data Leakage you must use only
#     #     working_train_df DataFrame to fit the OrdinalEncoder and
#     #     OneHotEncoder classes, then use the fitted models to transform all the
#     #     datasets.

#     # Taking the columns that contain objects.
#     category_columns = working_train_df.select_dtypes(include=['category']).columns

#     # Initializing the encoders
#     binary_encoder = OrdinalEncoder()
#     onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

#     # Filtering the dataset.
#     aux_dataframe = working_train_df[category_columns].copy()
#     mask_2 = (aux_dataframe.nunique() == 2).values
#     cat_2 = aux_dataframe.loc[:, mask_2].columns
#     mask_gt_2 = (aux_dataframe.nunique() > 2).values
#     cat_gt_2 = aux_dataframe.loc[:, mask_gt_2].columns

#     # Setting the tranform.
#     ct = ColumnTransformer(transformers=[
#         ('binary', binary_encoder, cat_2.tolist()),
#         ('onehot', onehot_encoder, cat_gt_2.tolist())
#     ], remainder='passthrough')

#     # Setting the dtype output
#     # ct.set_output(transform="pandas")

#     # fitting the ecoder.
#     ct.fit(working_train_df)

#     # Applying the transformation.
#     working_train_df = ct.transform(working_train_df)

#     working_val_df = ct.transform(working_val_df)

#     working_test_df = ct.transform(working_test_df)

#     # 3. TODO Impute values for all columns with missing data or, just all the columns.
#     # Use median as imputing value. Please use sklearn.impute.SimpleImputer().
#     # Again, take into account that:
#     #   - You must apply this to the 3 DataFrames (working_train_df, working_val_df,
#     #     working_test_df).
#     #   - In order to prevent overfitting and avoid Data Leakage you must use only
#     #     working_train_df DataFrame to fit the SimpleImputer and then use the fitted
#     #     model to transform all the datasets.

#     imputer = SimpleImputer(strategy='median', missing_values=np.nan)

#     imputer.fit(working_train_df)

#     # imputer.set_output(transform="pandas")

#     working_train_df = imputer.transform(working_train_df)

#     working_val_df = imputer.transform(working_val_df)

#     working_test_df = imputer.transform(working_test_df)

#     # 4. TODO Feature scaling with Min-Max scaler. Apply this to all the columns.
#     # Please use sklearn.preprocessing.MinMaxScaler().
#     # Again, take into account that:
#     #   - You must apply this to the 3 DataFrames (working_train_df, working_val_df,
#     #     working_test_df).
#     #   - In order to prevent overfitting and avoid Data Leakage you must use only
#     #     working_train_df DataFrame to fit the MinMaxScaler and then use the fitted
#     #     model to transform all the datasets.

#     scaler = MinMaxScaler()

#     scaler.fit(working_train_df)

#     working_train_df = scaler.transform(working_train_df)

#     working_val_df = scaler.transform(working_val_df)

#     working_test_df = scaler.transform(working_test_df)

#     return working_train_df, working_val_df, working_test_df

In [21]:
# app_train['FLAG_CARDS'] = np.where(app_train['FLAG_VISA'])

In [22]:
# let's fill null values with 0 to indicate those persons are in the first category
# app_normal['MONTHS_IN_RESIDENCE'] = app_normal['MONTHS_IN_RESIDENCE'].fillna(0)


In [23]:
out = app_normal['RESIDENCE_TYPE'].explode().value_counts()
print(out)

KeyError: 'RESIDENCE_TYPE'

In [ ]:
app_normal['QUANT_CARS'].unique()

array([0, 1], dtype=int64)

In [ ]:
app_normal.groupby('QUANT_CARS')['QUANT_CARS'].value_counts()

QUANT_CARS  QUANT_CARS
0           0             33193
1           1             16807
Name: QUANT_CARS, dtype: int64

In [ ]:
app_normal['QUANT_CARS'].isna().sum()

0

In [ ]:
app_normal['MONTHS_IN_THE_JOB'].isna().sum()

0

In [ ]:
app_normal['MONTHS_IN_THE_JOB'].unique()


array([ 0,  9,  2,  4,  1,  5, 15, 27, 19,  3, 14, 10, 30,  8,  7, 32, 18,
       26, 35, 12,  6], dtype=int64)

In [ ]:
app_normal.groupby('MONTHS_IN_THE_JOB')['MONTHS_IN_THE_JOB'].value_counts()
#DELETE

MONTHS_IN_THE_JOB  MONTHS_IN_THE_JOB
0                  0                    49924
1                  1                       15
2                  2                       18
3                  3                       12
4                  4                        4
5                  5                        6
6                  6                        2
7                  7                        2
8                  8                        2
9                  9                        1
10                 10                       1
12                 12                       2
14                 14                       2
15                 15                       1
18                 18                       2
19                 19                       1
26                 26                       1
27                 27                       1
30                 30                       1
32                 32                       1
35                 35                      

In [ ]:
app_normal['PROFESSION_CODE'].isna().sum()

7756

In [ ]:
app_normal['PROFESSION_CODE'].unique()

array([ 9., 11., nan,  0.,  2., 13.,  7., 12.,  6., 16.,  8., 10., 15.,
       17.,  3.,  4.,  5., 18., 14.,  1.])

In [ ]:
app_normal.groupby('TARGET_LABEL_BAD=1')['PROFESSION_CODE', 'TARGET_LABEL_BAD=1'].value_counts()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_17312\2475708111.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  app_normal.groupby('TARGET_LABEL_BAD=1')['PROFESSION_CODE', 'TARGET_LABEL_BAD=1'].value_counts()


TARGET_LABEL_BAD=1  PROFESSION_CODE
0                   9.0                22159
                    11.0                2741
                    0.0                 2652
                    2.0                 2012
                    12.0                 366
                    10.0                 312
                    16.0                 270
                    13.0                 249
                    7.0                  139
                    6.0                  108
                    8.0                   94
                    15.0                  46
                    17.0                  28
                    4.0                   19
                    3.0                   14
                    5.0                   12
                    1.0                    6
                    14.0                   5
                    18.0                   1
1                   9.0                 7933
                    0.0                  888
                   

In [ ]:
app_normal['OCCUPATION_TYPE'].isna().sum()

7313

In [ ]:
app_normal['OCCUPATION_TYPE'].unique()

array([ 4., nan,  5.,  2.,  1.,  0.,  3.])

In [ ]:
app_normal.groupby('TARGET_LABEL_BAD=1')['OCCUPATION_TYPE', 'TARGET_LABEL_BAD=1'].value_counts()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_17312\3408953595.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  app_normal.groupby('TARGET_LABEL_BAD=1')['OCCUPATION_TYPE', 'TARGET_LABEL_BAD=1'].value_counts()


TARGET_LABEL_BAD=1  OCCUPATION_TYPE
0                   2.0                12338
                    1.0                 6966
                    5.0                 5018
                    4.0                 4974
                    0.0                 2063
                    3.0                  241
1                   2.0                 4609
                    4.0                 2026
                    5.0                 1873
                    1.0                 1776
                    0.0                  725
                    3.0                   78
dtype: int64

In [ ]:
# OCCUPATION_TYPE replece nan for weighted values, changed to category

# Obtén la distribución de los datos, excluyendo los valores NaN
valid_values = [1 , 2 , 3 , 4, 5, 0]  # las categorías válidas
distribution = app_normal[app_normal['OCCUPATION_TYPE'].isin(valid_values)]['OCCUPATION_TYPE'].value_counts(normalize=True)

# Identifica los valores que están NaN.
missing = app_normal['OCCUPATION_TYPE'].isnull()

# Imputar los valores NaN de acuerdo a la distribución de los datos existentes.
app_normal.loc[missing, 'OCCUPATION_TYPE'] = np.random.choice(distribution.index, size=len(app_normal[missing]), p=distribution.values)
app_normal['OCCUPATION_TYPE'] = app_normal['OCCUPATION_TYPE'].astype('category')


In [ ]:
app_normal['OCCUPATION_TYPE'].isna().sum()

0

In [ ]:
app_normal['OCCUPATION_TYPE'].unique()

array([4., 5., 1., 2., 0., 3.])

In [ ]:
app_normal.groupby('OCCUPATION_TYPE')['OCCUPATION_TYPE'].value_counts()

OCCUPATION_TYPE  OCCUPATION_TYPE
0.0              0.0                 3277
1.0              1.0                10237
2.0              2.0                19837
3.0              3.0                  374
4.0              4.0                 8167
5.0              5.0                 8108
Name: OCCUPATION_TYPE, dtype: int64

In [ ]:
app_normal['EDUCATION_LEVEL'].isna().sum() 

0

In [ ]:
app_normal['EDUCATION_LEVEL'].unique()

C:\Users\Usuario\AppData\Roaming\Python\Python311\site-packages\IPython\lib\pretty.py:778: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  output = repr(obj)


[0.0, 2.0, 4.0, 5.0, 3.0, 1.0]
Categories (6, float64): [0.0, 1.0, 2.0, 3.0, 4.0, 5.0]

In [ ]:
app_normal.groupby('EDUCATION_LEVEL')['EDUCATION_LEVEL'].value_counts()

EDUCATION_LEVEL  EDUCATION_LEVEL
0.0              0.0                45278
                 1.0                    0
                 2.0                    0
                 3.0                    0
                 4.0                    0
                 5.0                    0
1.0              1.0                  158
                 0.0                    0
                 2.0                    0
                 3.0                    0
                 4.0                    0
                 5.0                    0
2.0              2.0                  971
                 0.0                    0
                 1.0                    0
                 3.0                    0
                 4.0                    0
                 5.0                    0
3.0              3.0                 1723
                 0.0                    0
                 1.0                    0
                 2.0                    0
                 4.0                    0
 

In [ ]:
app_normal['PRODUCT'].isna().sum() 

0

In [ ]:
app_normal['PRODUCT'].unique()

array([1, 2, 7], dtype=int64)

In [ ]:
app_normal['AGE'].isna().sum()

0

In [ ]:
app_normal['AGE'].unique()

array([ 32,  34,  27,  61,  48,  40,  28,  31,  41,  36,  29,  19,  49,
        58,  69,  39,  42,  47,  30,  26,  60,  45,  56,  71,  38,  53,
        79,  22,  37,  55,  25,  67,  21,  50,  51,  84,  43,  44,  57,
        33,  20,  63,  62,  24,  54,  64,  92,  65,  73,  91,  35,  76,
        80,  59,  46,  70,  66,  52,  23,  77,  68,  18,  74,  72,  75,
        78,  83,  81,  98,  97,  85,  90,  87,  82,  17,  86,  94,  89,
        88,  96, 106,  95,  99, 101,  93,   6, 100,   7,  14], dtype=int64)

In [ ]:
def categorize_age(age):
    if age <= 20:
        return '0-20 años'
    elif 20 < age <= 30:
        return '20-30 años'
    elif 30 < age <= 40:
        return '30-40 años'
    elif 40 < age <= 50:
        return '40-50 años'
    elif 50 < age <= 60:
        return '50-60 años'
    else:
        return '> 60 años'

# Aplicar la función a la columna
app_normal['AGE'] = app_normal['AGE'].apply(categorize_age)

# Obtén la distribución de los datos, excluyendo los valores NaN
valid_values = ['0-20 años', '20-30 años', '30-40 años' , '40-50 años' , '50-60 años' ,'> 60 años']  # las categorías válidas
distribution = app_normal[app_normal['AGE'].isin(valid_values)]['AGE'].value_counts(normalize=True)

# Identifica los valores que están NaN.
missing = app_normal['AGE'].isnull()

# Imputar los valores NaN de acuerdo a la distribución de los datos existentes.
app_normal.loc[missing, 'AGE'] = np.random.choice(distribution.index, size=len(app_normal[missing]), p=distribution.values)

In [ ]:
app_normal['AGE'].unique()

array(['30-40 años', '20-30 años', '> 60 años', '40-50 años', '0-20 años',
       '50-60 años'], dtype=object)